In [2]:
import os
import numpy as np
from skimage import io, color
from skimage.filters import threshold_otsu
from skimage.measure import regionprops
from scipy import ndimage

In [3]:
# Define paths for all datasets
base_path = "D:/work/ai&ml/projects/FNN-signature verification/temp/Dataset"
datasets = ['dataset1', 'dataset2', 'dataset3', 'dataset4']  # Your dataset folders

In [4]:
# Preprocessing and Feature Extraction Functions (as before)
def rgb2gray(img):
    # Check if the image already has a single channel (grayscale)
    if len(img.shape) == 2:  # Already grayscale
        return img
    return color.rgb2gray(img)  # Convert to grayscale if it's an RGB image

def greybin(img):
    blur_radius = 0.8
    img = ndimage.gaussian_filter(img, blur_radius)
    thres = threshold_otsu(img)
    binimg = img > thres
    return np.logical_not(binimg)

def preproc(img):
    gray = rgb2gray(img)
    binimg = greybin(gray)
    r, c = np.where(binimg)
    return binimg[r.min():r.max(), c.min():c.max()]

def Ratio(img):
    return np.sum(img) / img.size

def Centroid(img):
    rows, cols = np.nonzero(img)
    centroid = np.mean(np.stack([rows, cols], axis=1), axis=0)
    return centroid[0] / img.shape[0], centroid[1] / img.shape[1]

def EccentricitySolidity(img):
    props = regionprops(img.astype(int))
    return props[0].eccentricity, props[0].solidity

def SkewKurtosis(img):
    h, w = img.shape
    x = np.arange(w)
    y = np.arange(h)
    xp = np.sum(img, axis=0)
    yp = np.sum(img, axis=1)
    cx = np.sum(x * xp) / np.sum(xp)
    cy = np.sum(y * yp) / np.sum(yp)
    sx = np.sqrt(np.sum(((x - cx) ** 2) * xp) / np.sum(img))
    sy = np.sqrt(np.sum(((y - cy) ** 2) * yp) / np.sum(img))
    skewx = np.sum(xp * ((x - cx) ** 3)) / (np.sum(img) * sx ** 3)
    skewy = np.sum(yp * ((y - cy) ** 3)) / (np.sum(img) * sy ** 3)
    kurtx = np.sum(xp * ((x - cx) ** 4)) / (np.sum(img) * sx ** 4) - 3
    kurty = np.sum(yp * ((y - cy) ** 4)) / (np.sum(img) * sy ** 4) - 3
    return (skewx, skewy), (kurtx, kurty)


In [5]:
def extract_features(image_path):
    img = io.imread(image_path)
    img = preproc(img)
    ratio = Ratio(img)
    cent_y, cent_x = Centroid(img)
    eccentricity, solidity = EccentricitySolidity(img)
    skewx, skewy = SkewKurtosis(img)[0]
    kurtx, kurty = SkewKurtosis(img)[1]
    return [ratio, cent_y, cent_x, eccentricity, solidity, skewx, skewy, kurtx, kurty]

In [6]:
# Step 2: Prepare Dataset
def prepare_dataset():
    data = []
    labels = []
    
    # Loop over each dataset (dataset1, dataset2, etc.)
    for dataset in datasets:
        genuine_path = os.path.join(base_path, dataset, 'real')
        forged_path = os.path.join(base_path, dataset, 'forge')
        
        # Loop over each image in genuine and forged folder
        for img_name in os.listdir(genuine_path):
            img_path = os.path.join(genuine_path, img_name)
            data.append(extract_features(img_path))
            labels.append(1)  # Genuine = 1

        for img_name in os.listdir(forged_path):
            img_path = os.path.join(forged_path, img_name)
            data.append(extract_features(img_path))
            labels.append(0)  # Forged = 0

    return np.array(data), np.array(labels)


In [7]:
# Prepare the dataset
X, y = prepare_dataset()
X = X / np.max(X) 

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Step 4: Build and Compile Model (Neural Network)
def build_model(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),  # ✅ use shape instead of input_shape
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# Step 5: Train Model
def train_model(X, y):
    

    model = build_model(X.shape[1])
    
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              epochs=50,
              batch_size=16,
              callbacks=[early_stop],
              verbose=1)
    
    # ✅ Save using the newer .keras format
    model.save("signature_model.keras")
    print("✅ Model saved as 'signature_model.keras'")

In [10]:
# Step 6: Load and Predict on Batched Input (to avoid retracing warning)
def predict_signature_batch(image_paths):
    from tensorflow.keras.models import load_model
    model = load_model("signature_model.keras")

    # ✅ Compile after loading if evaluating
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    features = [extract_features(img_path) for img_path in image_paths]
    features = np.array(features)

    predictions = model.predict(features, batch_size=4, verbose=0)
    for i, pred in enumerate(predictions):
        label = "Genuine" if pred > 0.5 else "Forged"
        print(f"{image_paths[i]} ➜ {label}")

In [11]:
train_model(X, y)

Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.5069 - loss: 0.6938 - val_accuracy: 0.5139 - val_loss: 0.6938
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5286 - loss: 0.6929 - val_accuracy: 0.5833 - val_loss: 0.6913
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5587 - loss: 0.6905 - val_accuracy: 0.5694 - val_loss: 0.6896
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5619 - loss: 0.6880 - val_accuracy: 0.5486 - val_loss: 0.6890
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5419 - loss: 0.6903 - val_accuracy: 0.6042 - val_loss: 0.6860
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5458 - loss: 0.6872 - val_accuracy: 0.5972 - val_loss: 0.6841
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5553 - loss: 0.6849 - val_accuracy: 0.5972 - val_loss: 0.6822
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5221 - loss: 0.6848 - val_accuracy: 0.6111 - va

In [12]:
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model

# Evaluate the model
def evaluate_model(X_test, y_test):
    model = load_model('signature_model.keras')  # ✅ updated format
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # ensure metrics work
    y_pred = model.predict(X_test, batch_size=4)  # ✅ batch prediction to avoid retracing
    y_pred = (y_pred > 0.5).astype(int)
    print(classification_report(y_test, y_pred))

# Predict a single signature
def predict_signature(image_path):
    model = load_model('signature_model.keras')  # ✅ updated format
    features = np.array(extract_features(image_path)).reshape(1, -1)
    pred = model.predict(features, batch_size=1)  # ✅ batch_size added
    print(f"{image_path} ➜ {'Genuine' if pred[0][0] > 0.5 else 'Forged'}")

In [13]:
import inspect

with open("signature_utils.py", "w") as f:
    f.write(inspect.getsource(extract_features))

In [14]:
# After splitting your dataset
evaluate_model(X_val, y_val)

36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
              precision    recall  f1-score   support

           0       0.66      0.80      0.72        74
           1       0.72      0.56      0.63        70

    accuracy                           0.68       144
   macro avg       0.69      0.68      0.67       144
weighted avg       0.69      0.68      0.68       144



In [15]:
predict_signature("dataset/dataset2/real/00101001.png")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
dataset/dataset2/real/00101001.png ➜ Genuine


In [16]:
predict_signature("dataset/dataset1/forge/02100002.png")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
dataset/dataset1/forge/02100002.png ➜ Forged
